In [41]:

import os, json
import pandas as pd
from nltk import word_tokenize, pos_tag_sents
from langdetect import detect
import regex as re
import numpy as np
from nltk.corpus import stopwords
import pickle

### In this Notebook i've done the following :
___


    1. loaded the the json data into a dataframe containing 3 columns ['DATE' 'TEXT' 'FOLDER' 'FILE' 'NAME']
    
    2. Done some classical text cleaning ( punctuation, lower casing, special caracters ...) and stored 
       the processed data in a new column
       
    3. Used the library langdetect to separate german texts from english ones.The library have high accuracy results,
       however some texts contains bot german and english so the decision is based on the word counts of each language.
       Then i created  a new column 'lang'. 
       
    4. I separated the dataframe into 2 others json-data_en and json_data_de
    
    5. In this step i removed stopwords from the 2 dataframes 
    
    6. Finaly i pickled ( serialized ) the 2 dataframes for futre use. They will be attached to the notebooks. 
    
    7. The Notebook Descriptif analysis and Topic Modeling TASA use this dataframes for vizualisations and topic modeling. 

___




In [9]:
os.listdir('C:/Users/amine/Bureau/Moritz-Topic_Modeling/TASA-and-Central-Bank-Communication-master/TASA-and-Central-Bank-Communication-master/Data/Bundesbank/Data')

['.DS_Store', 'Interviews', 'Monatsberichte', 'Pressenotizen', 'Reden']

In [10]:
# Path to directory where folders containing json files are
path = 'C:/Users/amine/Bureau/Moritz-Topic_Modeling/TASA-and-Central-Bank-Communication-master/TASA-and-Central-Bank-Communication-master/Data/Bundesbank/Data'

contents = [x for x in os.listdir(path) if not os.path.isfile(os.path.join(path,x))]

# List of all files in a list of tuples (folder, filename)
json_files = []
for x in contents:
    files = os.listdir(os.path.join(path, x))
    for file in files:
        if file.endswith('.json'):
            json_files.append((x, file))



In [11]:
#Create dataframe for files
df = pd.DataFrame(data=json_files, columns=["Filetype", "Filename"])

#get first and last files
df.head(10).append(df.tail(10))

,Filetype,Filename
0,Interviews,interviews_02_10_2018_13.json
1,Interviews,interviews_04_04_2019_8.json
2,Interviews,interviews_04_11_2018_12.json
3,Interviews,interviews_09_07_2019_3.json
4,Interviews,interviews_09_11_2018_11.json
5,Interviews,interviews_10_09_2018_17.json
6,Interviews,interviews_12_07_2019_2.json
7,Interviews,interviews_13_05_2019_6.json
8,Interviews,interviews_13_11_2018_10.json
9,Interviews,interviews_15_04_2019_7.json


In [12]:
# Pandas dataframe that will have the contents of file DATE and TEXT along with Folder and name

jsons_data = pd.DataFrame(columns=['DATE', 'TEXT', 'FOLDER', 'FILE NAME'])

for i, file in enumerate(json_files):
    with open(os.path.join(path, file[0], file[1]),'r', encoding='utf-8') as fd:
        json_text = json.load(fd)
        
        DATE = json_text['DATE']
        TEXT = json_text['TEXT'] if json_text['TEXT'] != '-' else '\n'.join(json_text['PDF_TEXT'])
        
        # Push a list of data into a pandas DataFrame at row given by 'index'
        jsons_data.loc[i] = [DATE, TEXT, file[0], file[1]]
        
jsons_data

,DATE,TEXT,FOLDER,FILE NAME
0,02.10.2018,Die Bundesbank digitalisiert die Bankenaufsich...,Interviews,interviews_02_10_2018_13.json
1,04.04.2019,„Europa sollte nicht nur zuschauen“\n\nIntervi...,Interviews,interviews_04_04_2019_8.json
2,04.11.2018,„Banken haben ausreichend Puffer“\n\nInterview...,Interviews,interviews_04_11_2018_12.json
3,09.07.2019,„Banken müssen jetzt handeln“\n\nInterview in ...,Interviews,interviews_09_07_2019_3.json
4,09.11.2018,Im Dienste ihrer Stabilität\n\nInterview mit d...,Interviews,interviews_09_11_2018_11.json
5,10.09.2018,Wuermeling: Banken haben bei Sicherheit noch L...,Interviews,interviews_10_09_2018_17.json
6,12.07.2019,„Die Aufsicht muss ein Auge auf Libra haben“\n...,Interviews,interviews_12_07_2019_2.json
7,13.05.2019,„Minuszinsen sind nur ein Teil der Probleme“\n...,Interviews,interviews_13_05_2019_6.json
8,13.11.2018,„Wir brauchen europäische Lösungen“\n\nIntervi...,Interviews,interviews_13_11_2018_10.json
9,15.04.2019,Wieso verschwindet der Fünfhunderter?\n\nInter...,Interviews,interviews_15_04_2019_7.json


In [15]:
def preprocess(txt):
    txt = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", txt)
    txt = txt.lower()
    txt = re.sub(r"\W"," ",txt)
    txt = re.sub(r"\d"," ",txt)
    txt = re.sub(r"\s+[a-z]\s+"," ",txt)
    txt = re.sub(r"\s+[a-z]$"," ",txt)
    txt = re.sub(r"^[a-z]\s+"," ",txt)
    txt = re.sub(r"\s+"," ",txt) 
    return txt

In [16]:
jsons_data['Processed TEXT']=jsons_data['TEXT'].apply(lambda txt: preprocess(txt))

 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text pr

 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text pr

 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text pr

 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text pr

 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text processed
 Text pr

In [25]:
for index, row in jsons_data.iterrows():
    if row['Processed TEXT'] == " ":
        jsons_data.drop(index, inplace=True)

In [26]:
jsons_data['lang']=jsons_data['Processed TEXT'].apply(lambda x:detect(x))

def lang(lan):
    if lan not in ['de','en']:
        return 'de'
    else:
        return lan
        
jsons_data['lang']=jsons_data['lang'].apply(lambda x:lang(x))

In [27]:
jsons_data

,DATE,TEXT,FOLDER,FILE NAME,Processed TEXT,lang
0,02.10.2018,Die Bundesbank digitalisiert die Bankenaufsich...,Interviews,interviews_02_10_2018_13.json,die bundesbank digitalisiert die bankenaufsich...,de
1,04.04.2019,„Europa sollte nicht nur zuschauen“\n\nIntervi...,Interviews,interviews_04_04_2019_8.json,europa sollte nicht nur zuschauen interview i...,de
2,04.11.2018,„Banken haben ausreichend Puffer“\n\nInterview...,Interviews,interviews_04_11_2018_12.json,banken haben ausreichend puffer interview mit...,de
3,09.07.2019,„Banken müssen jetzt handeln“\n\nInterview in ...,Interviews,interviews_09_07_2019_3.json,banken müssen jetzt handeln interview in der ...,de
4,09.11.2018,Im Dienste ihrer Stabilität\n\nInterview mit d...,Interviews,interviews_09_11_2018_11.json,im dienste ihrer stabilität interview mit dem ...,de
5,10.09.2018,Wuermeling: Banken haben bei Sicherheit noch L...,Interviews,interviews_10_09_2018_17.json,wuermeling banken haben bei sicherheit noch lu...,de
6,12.07.2019,„Die Aufsicht muss ein Auge auf Libra haben“\n...,Interviews,interviews_12_07_2019_2.json,die aufsicht muss ein auge auf libra haben in...,de
7,13.05.2019,„Minuszinsen sind nur ein Teil der Probleme“\n...,Interviews,interviews_13_05_2019_6.json,minuszinsen sind nur ein teil der probleme in...,de
8,13.11.2018,„Wir brauchen europäische Lösungen“\n\nIntervi...,Interviews,interviews_13_11_2018_10.json,wir brauchen europäische lösungen interview m...,de
9,15.04.2019,Wieso verschwindet der Fünfhunderter?\n\nInter...,Interviews,interviews_15_04_2019_7.json,wieso verschwindet der fünfhunderter interview...,de


### Categorize text's by language

In [28]:
jsons_data.groupby('lang').count()

,DATE,TEXT,FOLDER,FILE NAME,Processed TEXT
lang,,,,,
de,2333,2333,2333,2333,2333
en,193,193,193,193,193


In [29]:
jsons_data_en = jsons_data.loc[jsons_data.lang=='en']
jsons_data_de = jsons_data.loc[jsons_data.lang=='de']

In [30]:
jsons_data_en.reset_index(inplace=True)
jsons_data_de.reset_index(inplace=True)

In [31]:
jsons_data_de.drop('index',inplace=True,axis=1)
jsons_data_en.drop('index',inplace=True,axis=1)

C:\Users\amine\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [32]:
jsons_data_en.head()

,DATE,TEXT,FOLDER,FILE NAME,Processed TEXT,lang
0,19.05.2005,Pressenotiz ... Deutsche Bundesbank Ł Ext...,Pressenotizen,pressenotizen_19_05_2005_1747.json,pressenotiz deutsche bundesbank ł externe komm...,en
1,20.02.1999,Communiqué of G-7 Finance Ministers and Centra...,Pressenotizen,pressenotizen_20_02_1999_2125.json,communiqué of finance ministers and central ba...,en
2,25.05.1999,Deutsche BundesbankPressenotizPress release___...,Pressenotizen,pressenotizen_25_05_1999_2111.json,deutsche bundesbankpressenotizpress release___...,en
3,01.02.2008,Page 1 of 12 \n Deutsche Bundesbank Ł Communic...,Reden,reden_01_02_2008_790.json,page of deutsche bundesbank ł communications d...,en
4,01.03.2017,Current Developments in the Euro Area\n\nSpeec...,Reden,reden_01_03_2017_258.json,current developments in the euro area speech a...,en


In [33]:
jsons_data_de.head()

,DATE,TEXT,FOLDER,FILE NAME,Processed TEXT,lang
0,02.10.2018,Die Bundesbank digitalisiert die Bankenaufsich...,Interviews,interviews_02_10_2018_13.json,die bundesbank digitalisiert die bankenaufsich...,de
1,04.04.2019,„Europa sollte nicht nur zuschauen“\n\nIntervi...,Interviews,interviews_04_04_2019_8.json,europa sollte nicht nur zuschauen interview i...,de
2,04.11.2018,„Banken haben ausreichend Puffer“\n\nInterview...,Interviews,interviews_04_11_2018_12.json,banken haben ausreichend puffer interview mit...,de
3,09.07.2019,„Banken müssen jetzt handeln“\n\nInterview in ...,Interviews,interviews_09_07_2019_3.json,banken müssen jetzt handeln interview in der ...,de
4,09.11.2018,Im Dienste ihrer Stabilität\n\nInterview mit d...,Interviews,interviews_09_11_2018_11.json,im dienste ihrer stabilität interview mit dem ...,de


In [34]:
counter=0
def remove_stopwords(text,language,processed,verbose):
    word_list = text.split() 
    filtered_words = [word for word in word_list if word not in stopwords.words(language)]
    text = ' '.join(filtered_words)
    global counter
    counter+=1
    if verbose==1 and counter % 10 == 0:
        print(f"processed {counter}/{len(processed)}")
    if counter >= len(processed):
        counter=0
    return text

In [36]:
jsons_data_en['Processed TEXT']=jsons_data_en['Processed TEXT'].apply(lambda text:remove_stopwords(text,'english',jsons_data_en['Processed TEXT'],verbose=1))

processed 10/193
processed 20/193
processed 30/193
processed 40/193
processed 50/193
processed 60/193
processed 70/193
processed 80/193
processed 90/193
processed 100/193
processed 110/193
processed 120/193
processed 130/193
processed 140/193
processed 150/193
processed 160/193
processed 170/193
processed 180/193
processed 190/193


C:\Users\amine\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
jsons_data_de['Processed TEXT']=jsons_data_de['Processed TEXT'].apply(lambda text:remove_stopwords(text,'german',jsons_data_de['Processed TEXT'],verbose=1))

processed 10/2333
processed 20/2333
processed 30/2333
processed 40/2333
processed 50/2333
processed 60/2333
processed 70/2333
processed 80/2333
processed 90/2333
processed 100/2333
processed 110/2333
processed 120/2333
processed 130/2333
processed 140/2333
processed 150/2333
processed 160/2333
processed 170/2333
processed 180/2333
processed 190/2333
processed 200/2333
processed 210/2333
processed 220/2333
processed 230/2333
processed 240/2333
processed 250/2333
processed 260/2333
processed 270/2333
processed 280/2333
processed 290/2333
processed 300/2333
processed 310/2333
processed 320/2333
processed 330/2333
processed 340/2333
processed 350/2333
processed 360/2333
processed 370/2333
processed 380/2333
processed 390/2333
processed 400/2333
processed 410/2333
processed 420/2333
processed 430/2333
processed 440/2333
processed 450/2333
processed 460/2333
processed 470/2333
processed 480/2333
processed 490/2333
processed 500/2333
processed 510/2333
processed 520/2333
processed 530/2333
pr

C:\Users\amine\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
with open('jsons_data_de.pickle','wb') as f:
    pickle.dump(jsons_data_de,f)
    
with open('jsons_data_en.pickle','wb') as f:
    pickle.dump(jsons_data_en,f)